<a href="https://colab.research.google.com/github/yuchen170/GDSC_AI/blob/main/GDSC_AI%E7%A4%BE%E8%AA%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1
* import libraries

In [ ]:
# Step 1.1 code


* 載入資料集

In [ ]:
from google.colab import files
files.upload() #上傳kaggle.json

In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json # 讀取和執行
!kaggle competitions download -c digit-recognizer

In [ ]:
import zipfile
zip_train = zipfile.ZipFile('/content/digit-recognizer.zip', 'r')
zip_train.extractall('/content') # Extract the folder into content folder
zip_train.close()

# Step 2
* Data Visulization

In [ ]:
data = pd.read_csv('/content/train.csv')
# Step 2 code


* 資料太多了，為了讓後面訓練速度快一點這裡把資料減少一半
* 大家之後也可以把這段拿掉練習！

In [ ]:
class_counts = data['label'].value_counts()
reduce_count = class_counts.min() // 2

reduced_data = pd.DataFrame(columns=data.columns)

for label in class_counts.index:
    class_data = data[data['label'] == label]
    sampled_data = class_data.sample(n=reduce_count, random_state=42)
    reduced_data = reduced_data.append(sampled_data, ignore_index=True)

In [ ]:
class_counts = reduced_data['label'].value_counts()
print(class_counts)

# Step 3
* Split the data to training and validation

In [ ]:
# Step 3 code

# Step 4
* 轉換資料型態

In [ ]:
# Step 4 code

In [ ]:
plt.figure(figsize=(15,15))
i = 0
for image, label in tf_train_data.take(8):
    plt.subplot(1,8,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(image.numpy().reshape((28, 28)), cmap='gray')
    plt.xlabel(label.numpy())

    i += 1

# Step 5 Data Preprocessing
* 資料的預處理，讓訓練速度快又精準的關鍵！

In [ ]:
# Step 5.1 code

In [ ]:
# Step 5.2 code

# Step 6 定義模型

In [ ]:
# Step 6 code

# Step 7 Compile

In [ ]:
model.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 8 訓練

In [ ]:
# Step 8.1 code

In [ ]:
# Step 8.2 code

# Step 9 訓練結果

In [ ]:
plt.plot(hist.history['accuracy'], label='accuracy')
plt.plot(hist.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

print('Training accuracy: %f' % hist.history['accuracy'][-1])
print('Validation accuracy: %f' % hist.history['val_accuracy'][-1])

In [ ]:
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

print('Training loss: %f' % hist.history['loss'][-1])
print('Validation loss: %f' % hist.history['val_loss'][-1])

# Step 10 測試

In [ ]:
test_data = pd.read_csv('/content/test.csv')
tf_test_data = tf.data.Dataset.from_tensor_slices(([test_data.to_numpy().reshape(len(test_data), 28, 28, 1)]))

In [ ]:
# Step 10 code


In [ ]:
plt.figure(figsize=(15,15))
i = 0
for i, row in test_data.head(20).iterrows():
    plt.subplot(4,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(row.values.reshape((28, 28)), cmap='gray')
    plt.xlabel(result[i])

# 測測看自己寫的字

In [ ]:
from google.colab import files
files.upload() #上傳larger.zip

In [ ]:
zip_train = zipfile.ZipFile('/content/larger.zip', 'r') #Opens the zip file in read mode
zip_train.extractall('/content') #Extracts the files into the /tmp folder
zip_train.close()

In [ ]:
image_folder = '/content/larger'  # 資料夾的名稱

image_data = np.empty((0, 28 * 28), dtype=np.uint8)
labels = np.empty((0, 1), dtype=int)

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        image = Image.open(os.path.join(image_folder, filename)).convert('L')
        print(image)
        image_array = np.array(image).flatten()
        print(len(image_array))
        label = int(filename.split(".")[0])
        labels = np.vstack((labels, label))
        image_data = np.vstack((image_data, image_array))

df = pd.DataFrame(data=image_data, columns=reduced_data.columns[1:785])

In [ ]:
df_test_data = tf.data.Dataset.from_tensor_slices(([df.to_numpy().reshape(len(df), 28, 28, 1)]))

predictions = model.predict(df_test_data)
predictions = np.argmax(predictions, axis=1)

In [ ]:
plt.figure(figsize=(10,10))

for i, row in df.head(10).iterrows():
    plt.subplot(2,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

    plt.imshow(row.values.reshape((28, 28)), cmap='gray')
    plt.xlabel(predictions[i])